In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models import KnownModelName
from pastel.classifiers import classify_images, classify_insurance_text, classify_insurance_image
from pastel.parsers import parse_assertion, parse_evidence, check_grammar
from pastel.evaluation import consolidate_evaluations

from pprint import pprint
import pandas as pd
import logfire
import os
from pastel.helpers import load_images_from_directory, create_consolidated_image
from pastel.models import (
    InsightModel,
    InsightValidation,
    InsightPlots,
    InputModel,
    PremiseValidation,
    BaseImage,
)
from pastel.prompts import EVALUATION_PROMPT, CLAIMS_VALIDATION_PROMPT

logfire.configure(token=os.getenv("PYDANTIC_LOGFIRE_TOKEN"))
logfire.instrument_openai()
logfire.instrument_anthropic()

OPENAI_MODEL: KnownModelName = "openai:gpt-4o-mini"
ANTHROPIC_MODEL: KnownModelName = "anthropic:claude-3-5-sonnet-latest"

index = 4

In [ ]:
insights = pd.read_excel("../data/Insights.xlsx")

input_model = InputModel(
    name=insights["programname"][index],
    insight=insights["insight"][index],
    line_of_business=insights["line_of_business"][index],
)

assertion = await parse_assertion(input_model)
insight = await parse_evidence(assertion)

In [4]:
# Retrieve data sets
lrs = pd.read_excel("../data/lrs.xlsx")
lrs_data = lrs.loc[lrs["programname"] == insight.name].drop(columns=["programname"])

In [ ]:
insight.name

In [ ]:
images = InsightPlots(plots=load_images_from_directory(f"../data/{insight.name.replace('/', '-')}"))
images

In [7]:
# Example usage
consolidated_image = create_consolidated_image(lrs_data, images, insight)

In [ ]:
consolidated_image

In [ ]:
insight

In [ ]:
premises = await classify_insurance_image(consolidated_image, insight.evidence)
premises

In [ ]:
grammar = await check_grammar(insight)
grammar

In [ ]:
final_evaluation = await consolidate_evaluations(insight, premises, grammar)